In [ ]:
import pandas as pd
import numpy as np
import PIL.Image as Image
import torch
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import cv2
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
import random
%matplotlib inline

In [ ]:
# !unzip '/content/Testing.zip'

In [ ]:
# !unzip '/content/Training.zip'

In [ ]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

In [ ]:
df_test = pd.read_csv("/kaggle/input/testing/driving_log.csv")

In [ ]:
df_train= df = pd.read_csv("/kaggle/input/Training/driving_log_Training.csv")

In [ ]:
df_train.head()

In [ ]:
df_train['Reverse'].value_counts()

In [ ]:
df_train.drop(columns=['Reverse'],inplace = True)

In [ ]:
img = mpimg.imread('/kaggle/input/Training/IMG_Training/center_2023_09_28_15_15_44_166.jpg')

# Display the image
plt.imshow(img)
plt.axis('off')  # Turn off axis labels and ticks (optional)
plt.show()


In [ ]:
df_train['Centre Image'] = df_train['Centre Image'].str.replace('D:\\IMG\\', '/kaggle/input/Training/IMG_Training/')
df_train['Left Image'] = df_train['Left Image'].str.replace('D:\\IMG\\', '/kaggle/input/Training/IMG_Training/')
df_train['Right Image'] = df_train['Right Image'].str.replace('D:\\IMG\\', '/kaggle/input/Training/IMG_Training/')

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df_test.columns = ['Centre Image' ,'Left Image', 	'Right Image', 	'Steering', 'Throttle' ,'Reverse','Speed']

In [ ]:
df_test['Reverse'].value_counts()

In [ ]:
df_test.drop(columns=['Reverse'],inplace = True)

In [ ]:
df_test['Centre Image'] = df_test['Centre Image'].str.replace('D:\\Self-Driving-Car Data\\IMG\\', '/kaggle/input/testing/IMG/')
df_test['Left Image'] = df_test['Left Image'].str.replace('D:\\Self-Driving-Car Data\\IMG\\', '/kaggle/input/testing/IMG/')
df_test['Right Image'] = df_test['Right Image'].str.replace('D:\\Self-Driving-Car Data\\IMG\\', '/kaggle/input/testing/IMG/')

In [ ]:
df_test.head()

## Concatenating all image columns

In [ ]:
new_df = pd.DataFrame(index=range(689*3), columns=range(1))
new_df_test = pd.DataFrame(index=range(460*3), columns=range(1))

new_df['All Images'] = df_train['Centre Image']
new_df_test['All Images'] = df_test['Centre Image']
new_df

In [ ]:
df_test['Right Image']

In [ ]:
new_df['All Images'].iloc[688: 688*2]=df_train['Right Image']
new_df_test['All Images'].iloc[460: 460*2]=df_test['Right Image']

In [ ]:
new_df['All Images'].iloc[688*2: 688*3]=df_train['Left Image']
new_df_test['All Images'].iloc[460*2: 460*3]=df_test['Left Image']

new_df

In [ ]:
new_df.drop(columns=[0],inplace=True)
new_df_test.drop(columns=[0],inplace=True)

In [ ]:
new_df.dropna(inplace=True)
new_df_test.dropna(inplace=True)

In [ ]:
new_df

In [ ]:
new_df['Steering'] = pd.Series()
new_df['Throttle'] = pd.Series()
new_df['Speed'] = pd.Series()

new_df_test['Steering'] = pd.Series()
new_df_test['Throttle'] = pd.Series()
new_df_test['Speed'] = pd.Series()

In [ ]:
new_df['Steering'] = df_train['Steering']
new_df['Throttle'] = df_train['Throttle']
new_df['Speed'] = df_train['Speed']

new_df_test['Steering'] = df_test['Steering']
new_df_test['Throttle'] = df_test['Throttle']
new_df_test['Speed'] = df_test['Speed']

In [ ]:
new_df['Steering'].iloc[688: 688*2]=df_train['Steering']
new_df['Throttle'].iloc[688: 688*2]=df_train['Throttle']
new_df['Speed'].iloc[688: 688*2]=df_train['Speed']

new_df_test['Steering'].iloc[460: 460*2]=df_test['Steering']
new_df_test['Throttle'].iloc[460: 460*2]=df_test['Throttle']
new_df_test['Speed'].iloc[460: 460*2]=df_test['Speed']

In [ ]:
new_df['Steering'].iloc[688*2: 688*3]=df_train['Steering']
new_df['Throttle'].iloc[688*2: 688*3]=df_train['Throttle']
new_df['Speed'].iloc[688*2: 688*3]=df_train['Speed']


new_df_test['Steering'].iloc[460*2: 460*3]=df_test['Steering']
new_df_test['Throttle'].iloc[460*2: 460*3]=df_test['Throttle']
new_df_test['Speed'].iloc[460*2: 460*3]=df_test['Speed']

In [ ]:
new_df.head()

In [ ]:
new_df.shape

In [ ]:
df_train= new_df.copy()
df_test= new_df_test.copy()

In [ ]:
df_train.head()

In [ ]:
df_test.head()

## Balancing Steering and Throttle

In [ ]:
import seaborn as sns
sns.kdeplot(df_train['Steering'])

In [ ]:
df_train['Steering'].value_counts()

In [ ]:
df_train['Throttle'].value_counts()

In [ ]:
import pandas as pd

# Your existing code
non_zero_mask = (df_train['Steering'] != 0)
zero_mask = (df_train['Steering'] == 0)
num_zero_samples = zero_mask.sum()
num_samples_to_keep = int(num_zero_samples * 0.2)

downsampled_zero_samples = df_train[zero_mask].sample(n=num_samples_to_keep, random_state=42)

df_train_downsampled = pd.concat([downsampled_zero_samples, df_train[non_zero_mask]])

df_train_downsampled= df_train_downsampled.reset_index()

In [ ]:
df_train_downsampled.drop(columns=['index'],inplace= True)
df_train_downsampled

In [ ]:
sns.kdeplot(df_train_downsampled['Steering'])

In [ ]:
df_train_downsampled['Steering'].value_counts()

In [ ]:
df_train = df_train_downsampled.copy()

In [ ]:
df_train.head()

In [ ]:
df_train2= df_train.copy()
df_test2= df_test.copy()

In [ ]:
df_test.shape

In [ ]:
df_train.shape

In [ ]:
sampled_df = df_test.sample(frac=0.4, random_state=42)
sampled_df.shape

In [ ]:
df_test= sampled_df.copy()

## Train, Test, Validation

In [ ]:
df_train.drop(columns=['Speed','Throttle'],inplace = True)
df_test.drop(columns=['Speed','Throttle'],inplace = True)

In [ ]:
df_train

In [ ]:
from sklearn.model_selection import train_test_split
train_data, val_data = train_test_split(df_train, test_size=0.2, random_state=42)

In [ ]:
from torch.utils.data import Dataset, DataLoader
class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx, 0]  
        image = Image.open(img_path)
        steering_angle = torch.tensor(float(self.dataframe.iloc[idx, 1]))  
        if self.transform:
            image = self.transform(image)

        return image, steering_angle


In [ ]:
import torchvision.transforms as transforms

class CustomCrop:
    def __init__(self, top_crop, bottom_crop):
        self.top_crop = top_crop
        self.bottom_crop = bottom_crop

    def __call__(self, img):
        # Get image size
        w, h = img.size

        # Crop the top and bottom
        img = img.crop((0, self.top_crop, w, h - self.bottom_crop))

        return img

transform = transforms.Compose([
    CustomCrop(top_crop=55, bottom_crop=25),
    transforms.Resize((66, 200)),
    transforms.ToTensor(),
#     transforms.Normalize(0, 255),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))


])



In [ ]:
batch_size = 32

train_dataset = CustomDataset(train_data, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,shuffle = True)

val_dataset = CustomDataset(val_data, transform=transform)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)

test_dataset = CustomDataset(df_test, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
import matplotlib.pyplot as plt

for batch_images, batch_steering_angles in train_loader:
    print("Batch shape:", batch_images.shape)
    print("Steering angles shape:", batch_steering_angles.shape)

    # Plot the first image in the batch
    plt.imshow(batch_images[1].permute(1, 2, 0))  # Assuming the images are in CHW format
    plt.show()

    break


## model_1

In [ ]:
# define the CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 66*200*3
        self.conv1 = nn.Conv2d(3, 24, 5)
        self.conv2 = nn.Conv2d(24, 36, 5)
        self.conv3 = nn.Conv2d(36, 48, 5)
        self.conv4 = nn.Conv2d(48, 64, 3)
        self.conv5 = nn.Conv2d(64, 64, 3)

        self.fc1 = nn.Linear(64 * 50 *184 , 100)
        self.fc2 = nn.Linear(100, 50)
        self.fc3= nn.Linear(50,10)
        self.fc4= nn.Linear(10,1)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = (F.relu(self.conv1(x)))
        x = (F.relu(self.conv2(x)))
        x = (F.relu(self.conv3(x)))
        x = (F.relu(self.conv4(x)))
        x = (F.relu(self.conv5(x)))
        # print("x shape is ",x.shape)

        x = x.view(-1, 64 * 50 * 184)
        x = self.dropout(x)

        x = F.relu(self.fc1(x))

        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x= (self.fc4(x))
        # print("x shape is ",x.shape)


        return x

In [ ]:
# create a complete CNN
model_1 = Net()
model_1

In [ ]:
if train_on_gpu:
    model_1.cuda()

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.SGD(model_1.parameters(), lr=0.01)

In [ ]:
n_epochs = 10

valid_loss_min = np.Inf # track change in validation loss
training_loss_list=[]
valid_loss_list=[]
for epoch in range(1, n_epochs+1):

    train_loss = 0.0
    valid_loss = 0.0

    model_1.train()
    for data, target in train_loader:
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model_1(data)
        # print("output.shape",output.shape)
        # print("target.shape",target.shape)
        loss = criterion(output,target.view(-1,1))
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)


    model_1.eval()
    for data, target in val_loader:
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        output = model_1(data)
        loss = criterion(output, target.view(-1,1))
        valid_loss += loss.item()*data.size(0)

    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(val_loader.sampler)

    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    training_loss_list.append(train_loss)
    valid_loss_list.append(valid_loss)

    if valid_loss < valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model_1 ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model_1.state_dict(), 'model_12.pt')
        valid_loss_min = valid_loss

In [ ]:
epochs= np.arange(1,11)
plt.figure(figsize=(8, 6))  # Adjust the figure size if needed
plt.plot(epochs, training_loss_list, label='Training Loss', color='blue')

In [ ]:
plt.plot(epochs, valid_loss_list, label='Validation Loss', color='red')

In [ ]:
epochs= np.arange(1,11)
plt.figure(figsize=(8, 6))  # Adjust the figure size if needed
plt.plot(epochs, training_loss_list, label='Training Loss', color='blue')
plt.plot(epochs, valid_loss_list, label='Validation Loss', color='red')

# Add labels and a legend
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Losses')
plt.legend()

In [ ]:
model_1.load_state_dict(torch.load('model_12.pt'))

In [ ]:
test_loss = 0.0
output_list = []
model_1.eval()


for data, target in test_loader:
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()

    output = model_1(data)
    output_list.append(output)
    loss = criterion(output, target.view(-1,1))

    test_loss += loss.item() * data.size(0)

test_loss = test_loss / len(test_loader.dataset)
print('Test Loss (MSE): {:.6f}\n'.format(test_loss))

mae = sum([torch.abs(pred - true[1]).mean().item() for pred, true in zip(output_list, test_loader.dataset)]) / len(output_list)
print('Mean Absolute Error (MAE): {:.6f}\n'.format(mae))


In [ ]:

# Assuming 'output_list' is a list of PyTorch tensors on the GPU (cuda:0)
output_array_list = []

for tensor in output_list:
    tensor_cpu = tensor.cpu()  # Move the tensor to CPU
    numpy_array = tensor_cpu.detach().numpy()  # Convert to NumPy array
    output_array_list.append(numpy_array)
output_array_list

In [ ]:
output_array_list= np.array(output_array_list)
output_array_list[0]

In [ ]:
for i in range(len(output_array_list)):
    output_array_list[i] = np.squeeze(output_array_list[i])
    
output_array_list

In [ ]:
output_array_list.shape

## Predicting Throttle from Steering and Speed using Machine Learning

In [ ]:
df_train= pd.read_csv("/kaggle/input/Training/driving_log_Training.csv")

In [ ]:
df_test= pd.read_csv("/kaggle/input/testing/driving_log.csv")

In [ ]:
df_train.drop(columns=['Centre Image','Right Image','Left Image','Reverse'],inplace= True)

In [ ]:
df_test.columns = ['Centre Image' ,'Left Image', 'Right Image', 'Steering', 'Throttle' ,'Reverse','Speed']

In [ ]:
df_test.drop(columns=['Centre Image','Right Image','Left Image','Reverse'],inplace= True)

In [ ]:
df_train.head()

In [ ]:
df_train['Throttle'].value_counts()

In [ ]:
sns.kdeplot(df_train['Throttle'])

In [ ]:

non_zero_mask = (df_train['Throttle'] != 1)
zero_mask = (df_train['Throttle'] == 1)
num_zero_samples = zero_mask.sum()
num_samples_to_keep = int(num_zero_samples * 0.2)

downsampled_zero_samples = df_train[zero_mask].sample(n=num_samples_to_keep, random_state=42)

df_train_downsampled = pd.concat([downsampled_zero_samples, df_train[non_zero_mask]])

df_train_downsampled= df_train_downsampled.reset_index()
df_train_downsampled.drop(columns=['index'],inplace= True)

In [ ]:
df_train_downsampled['Throttle'].value_counts()

In [ ]:

non_zero_mask = (df_train_downsampled['Throttle'] != 0)
zero_mask = (df_train_downsampled['Throttle'] == 0)
num_zero_samples = zero_mask.sum()
num_samples_to_keep = int(num_zero_samples * 0.2)

downsampled_zero_samples = df_train_downsampled[zero_mask].sample(n=num_samples_to_keep, random_state=42)

df_train_downsampled = pd.concat([downsampled_zero_samples, df_train_downsampled[non_zero_mask]])

df_train_downsampled= df_train_downsampled.reset_index()
df_train_downsampled.drop(columns=['index'],inplace= True)

In [ ]:
df_train_downsampled['Throttle'].value_counts()

In [ ]:
sns.kdeplot(df_train_downsampled['Throttle'])

In [ ]:
df_train = df_train_downsampled.copy()

In [ ]:
X_train = df_train[['Speed','Steering']]
X_test= df_test[['Speed','Steering']]
y_train = df_train['Throttle']
y_test = df_test['Throttle']

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

linreg = LinearRegression()
linreg.fit(X_train, y_train)

y_pred = linreg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print("R-squared:", r2)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)  # Adjust hyperparameters as needed
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print("R-squared:", r2)

In [ ]:
y_pred = rf.predict(X_test)

y_pred